In [2]:
import pandas as pd
import os, glob, re

from owlready2 import *
import owlready2
print(owlready2.VERSION)


import importlib.util
import sys
spec = importlib.util.spec_from_file_location("rdfutils", "../../../utils/rdfutils.py")
u = importlib.util.module_from_spec(spec)
sys.modules["rdfutils"] = u
spec.loader.exec_module(u)

spec = importlib.util.spec_from_file_location("llm", "../../../utils/llm.py")
h = importlib.util.module_from_spec(spec)
sys.modules["llm"] = h
spec.loader.exec_module(h)

from datetime import datetime

def NOW():
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    return "Current Time = "+ str(current_time)

%load_ext autoreload
%autoreload 2


0.40


* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [3]:
import requests, json

In [4]:
onto = get_ontology("./WIP_0.owl").load()
comments = u.checkComments(onto)

ID: 0 	 Description: Creation of a knowledge graph based on a litterature review, augmented by use of LLMs.
ID: 1 	 Version: 0.4
ID: 2 	 Library: owlready2==0.45
ID: 3 	 Changes from: 0.3
ID: 4 	 Next: Check formulas for selecting most repeated items in groups
ID: 5 	 Changes: Adding new risks, groups of items, and new relationships
ID: 6 	 Creation: 22/01/2024
ID: 7 	 TODOs: Adding synonyms and solving classification with synonyms
ID: 8 	 VersionComment: Grouping of items added in 0.4
ID: 9 	 Project: PROBONO
ID: 10 	 Next: Linking benefits to mitigations groups
ID: 11 	 License: CC BY-NC-SA
ID: 12 	 Task: T3.5
ID: 13 	 Repository: https://github.com/mm80843/T3.5/
ID: 14 	 Author: Luc Jonveaux
ID: 15 	 Language: English


In [5]:
with onto:
    class ISO_Issue(onto.PBNThing):
        label = ["ISO37100 Issue"]
        pass

In [6]:
dIDct = u.createDict(onto) 

error with the maxes


# Redo ISO

In [7]:
p = []
A = list(onto.RiskSubgroup.instances())
for a in A:
    p.append([a.name,a.has_Description[0].split("\n\nSources: ")[0].strip()])
A = list(onto.TechSubgroup.instances())
for a in A:
    p.append([a.name,a.has_Description[0].split("\n\nSources: ")[0].strip()])
A = list(onto.BP_Intervention.instances())
for a in A:
    if a.has_Description:
        p.append([a.name,a.has_Description[0].strip()])
    
df = pd.DataFrame(p,columns =["ID","Description"])
df.to_parquet("df_iso.parquet.gzip",compression="gzip")

In [8]:
dd = pd.read_parquet("df_iso_done.parquet.gzip")
dd.head(3)

,WP,Deliverable,LL,Description,Justification,Purpose,Issue,Scale,Score,Justification_Short,Source_Title,done,ID
0,WP0,DLV0,Unknown,"The term ""failure to implement preventive meas...",The text highlights the importance of preventi...,Resilience,Health and care in the community,Neighbourhood,5,Preventive measures in smart cities.,Failure to implement preventive measures risks...,auto,PBN__RiskSubgroup_2
1,WP0,DLV0,Unknown,Air filtering refers to the process of removin...,The text emphasizes the importance of air filt...,Preservation and improvement of environment,Health and care in the community,Building,5,Importance of air filtering for health,Air filtering for healthier environments.\n\n,auto,PBN__TechSubgroup_58
2,WP0,DLV0,Unknown,In the context of contagious diseases in smart...,The text emphasizes the role of ethical guidel...,Social cohesion,"Living together, interdependence and mutuality",Building,5,Ethical guidelines in managing contagious dise...,Ethical guidelines for disease control.,auto,PBN__TechSubgroup_44


In [9]:
with onto: 
    class has_ISOIssue(onto.PBNThing >> onto.ISO_Issue):
        label = ["ISO 37100 classification of issues"]
        pass
    class has_ISOPurpose(onto.PBNThing >> onto.ISO_Purpose):
        label = ["ISO 37100 classification of purpose"]
        pass
    class has_ISOScale(onto.PBNThing >> onto.ISO_Scale):
        label = ["ISO 37100 classification of scale"]
        pass
    class has_ISOJustif(onto.PBNThing >> str):
        label = ["ISO 37100 justification of category"]
        pass

In [10]:
for c in ["ISO_Impact","ISO_Scale","ISO_Purpose"]:
    b = eval("onto."+c)
    for k in b.instances():
        destroy_entity(k)

In [11]:
for ix, row in dd.iterrows():
    obj = eval("onto."+row["ID"])
    newPurpose =  u.addItem("ISO_Purpose", row["Purpose"], dIDct, onto )
    obj.has_ISOPurpose = []
    obj.has_ISOPurpose = [newPurpose]    
    newIssue  =  u.addItem("ISO_Issue", row["Issue"], dIDct, onto )
    obj.has_ISOImpact = []
    obj.has_ISOImpact = [newIssue]    
    newScale   =  u.addItem("ISO_Scale", row["Scale"], dIDct, onto )
    obj.has_ISOScale = []
    obj.has_ISOScale = [newScale]    

    obj.has_ISOJustif = []
    obj.has_ISOJustif = [row["Justification"]]    

onto.ISO_Purpose("PBN__ISO_Purpose_18")
onto.ISO_Issue("PBN__ISO_Issue_1")
onto.ISO_Scale("PBN__ISO_Scale_5")
onto.ISO_Purpose("PBN__ISO_Purpose_19")
onto.ISO_Scale("PBN__ISO_Scale_6")
onto.ISO_Issue("PBN__ISO_Issue_2")
onto.ISO_Issue("PBN__ISO_Issue_3")
onto.ISO_Issue("PBN__ISO_Issue_4")
onto.ISO_Issue("PBN__ISO_Issue_5")
onto.ISO_Issue("PBN__ISO_Issue_6")
onto.ISO_Purpose("PBN__ISO_Purpose_20")
onto.ISO_Issue("PBN__ISO_Issue_7")
onto.ISO_Issue("PBN__ISO_Issue_8")
onto.ISO_Issue("PBN__ISO_Issue_9")
onto.ISO_Issue("PBN__ISO_Issue_10")
onto.ISO_Purpose("PBN__ISO_Purpose_21")


# Cleaning Intervention

In [12]:
A = list(onto.BP_Intervention.instances())
print("Before:",len(A))
B = [a for a in A if "(" in a.label[0]]
PROP = [onto.has_InterventionBP,
 onto.has_Category,
 onto.has_BP_InterventionKey,
 onto.has_BP_InterventionTitle]
for b in B:
    src = b
    dest = [x for x in A if x.label[0] == b.label[0].replace("(","").replace(")","")][0]
    dest.has_InterventionBP = []
    for k in src.has_InterventionBP:
        dest.has_InterventionBP.append(k)
    dest.has_Category = []
    for k in src.has_Category:
        dest.has_Category.append(k)
    dest.has_BP_InterventionKey = []
    for k in src.has_BP_InterventionKey:
        dest.has_BP_InterventionKey.append(k)
    dest.has_BP_InterventionTitle = []
    for k in src.has_BP_InterventionTitle:
        dest.has_BP_InterventionTitle.append(k)

    destroy_entity(b)

A = list(onto.BP_Intervention.instances())
print("After:",len(A))

Before: 46
After: 40


# Relabeling

In [13]:
with onto:
    class has_Description(onto.PBNThing >> str):
        label = ["Short description"]
        pass
    class has_Note(onto.PBNThing >> str):
        label = ["Author note"]
        pass

In [14]:
onto.BenefReturn.label = ["Benefit returns"]
onto.Benef.label = ["Benefit"]
onto.Article.label = ["Research article"]
onto.Risk.label = ["Risk"]
onto.ISO_Scale.label = ["Scale in the ISO37100 (GBN) reference system"]
onto.ISO_Scale.has_Note = ["Used according to the ISO 37100 exercice to assess the scale an item in a GBN."]
onto.RiskHealth.label = ["Type of health impact"]
onto.RiskType.label = ["Type of the risk"]
onto.Stakeholder.label = ["Stakeholder"]
onto.Stakeholder_Type.label = ["Type of stakeholder"]
onto.Technology.label = ["Technology"]
onto.ISO_Impact.label = ["Impact in the ISO37100 (GBN) reference system"]
onto.ISO_Purpose.label = ["Purpose in the ISO37100 (GBN) reference system"]
onto.StakeholderGroup.label = ["Group of stakeholders"]
onto.StakeholderSubgroup.label = ["Subroup of stakeholders"]
onto.TechGroup.label = ["Group of technologies"]
onto.TechSubgroup.label = ["Subroup of technologies"]
onto.Mitigation.label = ["Risk mitigation"]
onto.BP_Enabler.label = ["Blueprint: enabler"]
onto.BP_Transmission.label = ["Blueprint: transmission mode"]
onto.Blueprint.label = ["Blueprint"]
onto.BP_Scale.label = ["Blueprint: scale"]
onto.BP_Phase.label = ["Blueprint: phase"]
onto.BP_Permanent.label = ["Blueprint: is it permanent?"]
onto.BP_Type.label = ["Blueprint: Type"]
onto.BP_Intervention.label = ["Blueprint: intervention family"]
onto.PBNCategory.label = ["Category of objects"]
onto.RiskGroup.label = ["Group of risks"]
onto.RiskSubgroup.label = ["Subgroup of risks"]
onto.CAO_Group.label = ["Localisation: City Anatomy level 1"]
onto.CAO_Group.has_Note = ["For use with the City Anatomy ontology.\n See page 26 of https://s3.amazonaws.com/nist-sgcps/smartcityframework/files/GCTCTechJamKickoff/JimAloisi_NIST_IES_city_presentation_jarecov_3_2016-2.pdf ."]
onto.CAO_Subgroup.label = ["Localisation: City Anatomy level 2"]
onto.CAO_Subgroup.has_Note = ["For use with the City Anatomy ontology.\n See page 26 of https://s3.amazonaws.com/nist-sgcps/smartcityframework/files/GCTCTechJamKickoff/JimAloisi_NIST_IES_city_presentation_jarecov_3_2016-2.pdf ."]
onto.BP_Theme.label = ["Blueprint: Theme"]
onto.BP_Category.label = ["Blueprint: Category"]
onto.aBlueprint.label = ["Blueprint, by genAI"]
onto.aBlueprint.has_Note = ["For inspiration only. Generated by AI, not reviewed by humans."]

In [15]:
A = list(onto.classes())[2]
len(A.instances())

1409

In [16]:
onto.has_BP_InterventionTitle.label = ["Intervention title"]
onto.has_BP_InterventionKey = ["Key/ID of the intervention"]

In [17]:
L = onto.PBNThing.instances()

In [18]:
nolabel = []
toremove = []
for l in L:
    if not l.label:
        print(l)
        nolabel.append(l)
    else:
        if l.label[0] == "Nan":
            print(l)
        if not len(l.label[0].replace("-","").replace("=","").strip()):
            toremove.append(l)
            print(l)

WIP_0.PBN__BenefReturn_146
WIP_0.PBN__BenefReturn_1109
WIP_0.PBN__BenefReturn_1111
WIP_0.PBN__BenefReturn_1113
WIP_0.PBN__BenefReturn_1115
WIP_0.PBN__BenefReturn_1117
WIP_0.PBN__BenefReturn_1119
WIP_0.PBN__BenefReturn_837
WIP_0.PBN__BenefReturn_1551
WIP_0.PBN__Benef_145
WIP_0.PBN__Benef_999
WIP_0.PBN__Benef_765
WIP_0.PBN__Benef_1359
WIP_0.PBN__Risk_295
WIP_0.PBN__Risk_715
WIP_0.PBN__Risk_2295
WIP_0.PBN__Risk_2260
WIP_0.PBN__Risk_536
WIP_0.PBN__Risk_1565
WIP_0.PBN__Risk_986
WIP_0.PBN__Risk_7874
WIP_0.PBN__RiskHealth_6
WIP_0.PBN__RiskHealth_13
WIP_0.PBN__RiskHealth_12
WIP_0.PBN__RiskHealth_10
WIP_0.PBN__RiskType_14
WIP_0.PBN__RiskType_21
WIP_0.PBN__RiskType_12
WIP_0.PBN__Stakeholder_83
WIP_0.PBN__Stakeholder_1543
WIP_0.PBN__Stakeholder_1519
WIP_0.PBN__Stakeholder_1014
WIP_0.PBN__Stakeholder_314
WIP_0.PBN__Stakeholder_396
WIP_0.PBN__Technology_22
WIP_0.PBN__Technology_3360
WIP_0.PBN__Technology_355
WIP_0.PBN__Technology_1865
WIP_0.PBN__Technology_1909
WIP_0.PBN__Technology_882
WIP_0.PBN__

In [19]:
for indiv in toremove :
    destroy_entity(indiv)
onto.PBN__Mitigation_2106

# Adding CAO

In [20]:
classify = [k for k in onto.classes() if ("roup" in str(k)) and (not "CAO" in str(k))]
print(classify)
N = classify[0].name
DFC = pd.read_parquet(N+"_newCAO_classified_lv1.parquet.gzip")
# remains at @TODO
DFC

[WIP_0.StakeholderGroup, WIP_0.StakeholderSubgroup, WIP_0.TechGroup, WIP_0.TechSubgroup, WIP_0.RiskGroup, WIP_0.RiskSubgroup]


,name,category,confidence,src,subcategory,confidence_lv2
0,Health organisations,Society,High,StakeholderGroup,Organizations,Medium-high
1,Research organisations,Interactions,Medium-high,StakeholderGroup,Working,Medium
2,Government institutions,Society,High,StakeholderGroup,Organizations,Medium-high
3,General public,Interactions,High,StakeholderGroup,Social Expression,Medium
4,Public entities,Interactions,Medium,StakeholderGroup,Commerce/Trade,Medium
5,Environmental and sustainability organizations,Interactions,High,StakeholderGroup,Social Expression,High
6,Workers,Interactions,Medium-high,StakeholderGroup,Working,High
7,Financial groups,Interactions,Medium,StakeholderGroup,Wealth production,High
8,Residents and occupants of spaces,Interactions,Medium-high,StakeholderGroup,Living,Medium-high
9,Construction and infrastructure industry,Structure,Medium-high,StakeholderGroup,Buildings/Blocks,Medium-high


In [21]:
with onto:
    class has_CAOGroup(onto.PBNThing >> onto.CAO_Group):
        label = ["City anatomy layer that can be covered"]
        pass
    class has_CAOSubgroup(onto.PBNThing >> onto.CAO_Subgroup):
        label = ["City anatomy sublayer that can be covered"]
        pass


In [22]:
for c in classify:
    N = c.name
    print("Doing:",N)
    DFC = pd.read_parquet(N+"_newCAO_classified_lv1.parquet.gzip")
    for ix, row in DFC.iterrows():
        obj = u.addItem(N, row["name"], dIDct, onto )
        cate = u.addItem("CAO_Group", row["category"], dIDct, onto )
        scat = u.addItem("CAO_Subgroup", row["subcategory"], dIDct, onto )
        obj.has_CAOGroup = []
        obj.has_CAOGroup.append(cate)
        obj.has_CAOSubgroup = []
        obj.has_CAOSubgroup.append(scat)

Doing: StakeholderGroup
Doing: StakeholderSubgroup
onto.CAO_Subgroup("PBN__CAO_Subgroup_48")
Doing: TechGroup
Doing: TechSubgroup
Doing: RiskGroup
Doing: RiskSubgroup


# Saving

In [23]:
onto.save("WIP_1.owl")

In [24]:
print("Done")

Done
